In [1]:
import torch
from transformers import BertJapaneseTokenizer, BertModel
import ipadic
from tqdm import tqdm

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 4-3
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [3]:
# text_listに格納
with open('data_src/wiki40b_with_emotion/wiki_40b_train_with_emotion.txt', 'r') as f_read:
    text_list = []
    for line in f_read:
        text_list.append(line)
print(len(text_list))

757744


In [4]:
import torch
from torch.utils.data import DataLoader
# 文章の符号化

dataset_for_loader = []
count = 0 # for debug
for text in tqdm(text_list):
    encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True
    )
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)
    # count += 1 # for debug
    # if count == 100:
    #     break

  7%|▋         | 51036/757744 [00:28<06:34, 1791.88it/s]


KeyboardInterrupt: 

In [ ]:
# DataLoader生成
loader = DataLoader(dataset_for_loader, batch_size=512)

In [ ]:
# DataLoader動作確認
for idx, batch in enumerate(loader):
    print("batch:", idx)
    print(batch['input_ids'].size())

In [ ]:
bert = BertModel.from_pretrained(model_name)

bert = bert.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from my_module.tools import get_dataset_from_batch
count = 0
with torch.no_grad():
    # データセット出力先を指定
    with open('/workspace/emotion_analysis_test/nagasawa/data_src/BERT_to_emotion/only_emotion/BERT_to_emo_train.txt', 'a') as f:
        for batch in tqdm(loader):
            # データをGPUに乗せる
            batch = {k: v.cuda() for k, v in batch.items()}
            # BERTでの処理
            output = bert(**batch)
            last_hidden_state = output.last_hidden_state
            last_hidden_state = last_hidden_state.cpu()
            last_hidden_state = last_hidden_state.numpy()
            last_hidden_state = last_hidden_state.tolist()
            get_dataset_from_batch(batch, last_hidden_state, f)
            count += 1
            # if count == 3:
            #     break
print('done!')

0it [00:00, ?it/s]

done!
